In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import src.config as config 

In [3]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/878742
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/878742/fs/874565/fv/time_series_hourly_feature_view/version/4


In [5]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using Hive (1339.25s) 


In [6]:
# drop `pickup_ts` column
ts_data.drop('pickup_ts', axis=1, inplace=True)

# sort by `pickup_location_id` and `pickup_hour`
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data


,pickup_hour,rides,pickup_location_id
356991,2022-01-01 03:00:00,0,1
2057690,2022-01-01 04:00:00,1,1
216557,2022-01-01 05:00:00,1,1
614752,2022-01-01 06:00:00,0,1
1715383,2022-01-01 07:00:00,2,1
...,...,...,...
1025355,2024-07-16 01:00:00,4,265
1626687,2024-07-16 02:00:00,0,265
1104530,2024-07-16 03:00:00,3,265
1103869,2024-07-16 04:00:00,0,265


In [7]:
# from src.plot import plot_ts
from typing import Optional, List
import pandas as pd
import plotly.express as px 

def plot_ts(
    ts_data: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    ts_data_to_plot = ts_data[ts_data.pickup_location_id.isin(locations)] if locations else ts_data

    fig = px.line(
        ts_data_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

plot_ts(ts_data, locations=[43])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
d:\ML System 01\Taxi_Demand_Predictor\.venv\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [8]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

  0%|          | 0/255 [00:00<?, ?it/s]d:\ML System 01\Taxi_Demand_Predictor\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

  0%|          | 1/255 [00:00<01:13,  3.43it/s]d:\ML System 01\Taxi_Demand_Predictor\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

  1%|          | 2/255 [00:00<01:40,  2.52it/s]d:\ML System 01\Taxi_Demand_Predictor\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/de

features_and_target.shape=(105314, 675)


In [9]:
features_and_target

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,5.0,5.0,3.0,...,0.0,2.0,0.0,4.0,0.0,0.0,0.0,2022-03-01 02:00:00,1,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,0.0,...,0.0,0.0,1.0,1.0,2.0,0.0,2.0,2022-03-02 14:00:00,1,1.0
2,0.0,0.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2022-03-04 14:00:00,1,3.0
3,0.0,0.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2022-03-06 02:00:00,1,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,...,0.0,1.0,0.0,2.0,1.0,1.0,2.0,2022-03-07 22:00:00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105309,2.0,1.0,5.0,2.0,4.0,7.0,4.0,3.0,5.0,3.0,...,0.0,1.0,4.0,4.0,1.0,4.0,5.0,2024-07-11 11:00:00,265,6.0
105310,1.0,1.0,1.0,2.0,1.0,0.0,3.0,4.0,1.0,3.0,...,1.0,4.0,1.0,1.0,6.0,5.0,3.0,2024-07-12 10:00:00,265,3.0
105311,0.0,1.0,2.0,2.0,5.0,5.0,0.0,0.0,2.0,2.0,...,7.0,5.0,4.0,1.0,2.0,2.0,1.0,2024-07-13 09:00:00,265,3.0
105312,4.0,1.0,1.0,3.0,1.0,2.0,5.0,5.0,0.0,2.0,...,2.0,1.0,6.0,5.0,1.0,1.0,3.0,2024-07-14 08:00:00,265,2.0


In [10]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2022 up until 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1), utc=True)

print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2024-06-18 00:00:00+0000', tz='UTC')
X_train.shape=(98183, 674)
y_train.shape=(98183,)
X_test.shape=(7131, 674)
y_test.shape=(7131,)


In [11]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = KFold(n_splits=3)
    scores = []
        
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
         
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [16]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2024-07-16 12:06:50,547] A new study created in memory with name: no-name-2f88f08b-ed01-4aa5-a408-dbadf58f3429
D:\ML System 01\Taxi_Demand_Predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

D:\ML System 01\Taxi_Demand_Predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

D:\ML System 01\Taxi_Demand_Predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [17]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 148, 'feature_fraction': 0.8655813570824598, 'bagging_fraction': 0.931458948827143, 'min_child_samples': 10}


In [18]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.8655813570824598, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8655813570824598
[LightGBM] [Warning] bagging_fraction is set=0.931458948827143, subsample=1.0 will be ignored. Current value: bagging_fraction=0.931458948827143
[LightGBM] [Warning] feature_fraction is set=0.8655813570824598, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8655813570824598
[LightGBM] [Warning] bagging_fraction is set=0.931458948827143, subsample=1.0 will be ignored. Current value: bagging_fraction=0.931458948827143
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.100717 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171892
[LightGBM] [Info] Number of data points in the train set: 98183, number of used features: 676
[LightGBM] [Info] Start training from score 18.440647


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000001FC9375BE50>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.931458948827143,
                               feature_fraction=0.8655813570824598,
                               min_child_samples=10, num_leaves=148))])

In [19]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.8655813570824598, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8655813570824598
[LightGBM] [Warning] bagging_fraction is set=0.931458948827143, subsample=1.0 will be ignored. Current value: bagging_fraction=0.931458948827143
test_mae=3.9331


In [20]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['D:\\ML System 01\\Taxi_Demand_Predictor\\models\\model.pkl']

In [21]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [22]:
model_registry = project.get_model_registry()


model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save(str(MODELS_DIR / 'model.pkl'))

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/1388514 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/3399 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/60849 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/878742/models/taxi_demand_predictor_next_hour/1


Model(name: 'taxi_demand_predictor_next_hour', version: 1)